# TPS Transit Safety Case Competition - Station Standardization
## Prompt 2: Master Station List Creation

**Objective:** Create a unified master station list by intelligently merging ridership and coordinate data

**Key Challenges:**
- Station name mismatch: 86% overlap (63/73 matched)
- Duplicate stations (Bloor-Yonge, St. George on multiple lines)
- Ridership has 74 records, Stations has 73 unique
- Need fuzzy matching for name variants ("St. George" vs "ST GEORGE")

**Author:** Data Science Team

**Date:** January 24, 2026

---

## 1. Setup & Imports

In [65]:
# Standard libraries
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from datetime import datetime
import re
from difflib import SequenceMatcher
import math

# Suppress warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")

✓ Libraries imported successfully
Pandas version: 2.3.0


## 2. Configuration & Constants

In [66]:
from pathlib import Path

# Notebook is inside: TPS_CaseComp/modules/
PROJECT_ROOT = Path.cwd().parent

DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "outputs"

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Input files
STATIONS_PATH = DATA_DIR / "subway-stations.csv"
RIDERSHIP_PATH = DATA_DIR / "ttc-ridership.csv"

# Output files
MASTER_STATIONS_PATH = OUTPUT_DIR / "02_master_station_list.csv"
MATCHING_REPORT_PATH = OUTPUT_DIR / "02_name_matching_report.txt"
UNMATCHED_STATIONS_PATH = OUTPUT_DIR / "02_unmatched_stations.csv"

# Venue coordinates (from competition brief and Google Maps)
VENUES = {
    'BMO_FIELD': {'lat': 43.6332, 'lon': -79.4189, 'name': 'BMO Field (Exhibition Place)'},
    'SCOTIABANK_ARENA': {'lat': 43.6435, 'lon': -79.3791, 'name': 'Scotiabank Arena'},
    'ROGERS_CENTRE': {'lat': 43.6414, 'lon': -79.3894, 'name': 'Rogers Centre'}
}

# Proximity threshold (2km as specified)
PROXIMITY_THRESHOLD_KM = 2.0

# Fuzzy matching threshold
FUZZY_MATCH_THRESHOLD = 0.85

print("✓ Configuration loaded")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Venue proximity threshold: {PROXIMITY_THRESHOLD_KM} km")

✓ Configuration loaded
Output directory: /Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/TPS_CaseComp/outputs
Venue proximity threshold: 2.0 km


## 3. Utility Functions

In [67]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points on Earth.
    Returns distance in kilometers.
    
    Formula: Haversine distance
    Accuracy: ~0.5% error (sufficient for our use case)
    """
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Earth radius in kilometers
    r = 6371
    
    return c * r

def standardize_station_name(name):
    """
    Standardize station names for consistent matching.
    
    Rules:
    1. Convert to uppercase
    2. Remove extra spaces
    3. Standardize separators (hyphen vs space)
    4. Remove periods and apostrophes
    5. Handle special cases (St. → SAINT, etc.)
    """
    if pd.isna(name):
        return None
    
    # Convert to string and uppercase
    name = str(name).upper().strip()
    
    # Remove periods and apostrophes
    name = name.replace('.', '').replace("'", '')
    
    # Standardize "Saint" variations
    name = re.sub(r'\bST\s+', 'ST ', name)  # Ensure single space after ST
    
    # Remove extra whitespace
    name = re.sub(r'\s+', ' ', name)
    
    # Trim
    name = name.strip()
    
    return name

def fuzzy_match_stations(name1, name2):
    """
    Calculate similarity ratio between two station names.
    Returns value between 0 and 1 (1 = perfect match).
    
    Uses SequenceMatcher (similar to Levenshtein distance).
    """
    if pd.isna(name1) or pd.isna(name2):
        return 0.0
    
    name1 = standardize_station_name(name1)
    name2 = standardize_station_name(name2)
    
    return SequenceMatcher(None, name1, name2).ratio()

def find_best_match(target_name, candidate_names, threshold=FUZZY_MATCH_THRESHOLD):
    """
    Find the best matching station name from a list of candidates.
    
    Returns: (best_match, similarity_score) or (None, 0) if no good match
    """
    best_match = None
    best_score = 0
    
    for candidate in candidate_names:
        score = fuzzy_match_stations(target_name, candidate)
        if score > best_score:
            best_score = score
            best_match = candidate
    
    if best_score >= threshold:
        return best_match, best_score
    else:
        return None, best_score

print("✓ Utility functions defined")

# Test haversine distance
# Union Station to BMO Field should be ~4.7 km
union_lat, union_lon = 43.6452, -79.3806
bmo_lat, bmo_lon = 43.6332, -79.4189
test_dist = haversine_distance(union_lat, union_lon, bmo_lat, bmo_lon)
print(f"\n✓ Distance calculation test: Union to BMO Field = {test_dist:.2f} km (expected ~4.7 km)")

✓ Utility functions defined

✓ Distance calculation test: Union to BMO Field = 3.36 km (expected ~4.7 km)


## 4. Load Data

In [68]:
print("Loading datasets...")

# Load stations data
stations_df = pd.read_csv(STATIONS_PATH, encoding='utf-8-sig')
print(f"✓ Loaded {len(stations_df)} station records")

# Load ridership data
ridership_df = pd.read_csv(RIDERSHIP_PATH)
print(f"✓ Loaded {len(ridership_df)} ridership records")

# Identify key columns from stations data (they have long prefixes)
station_name_col = [col for col in stations_df.columns if 'PT_NAME' in col][0]
lat_col = [col for col in stations_df.columns if 'LATITUDE' in col][0]
long_col = [col for col in stations_df.columns if 'LONGITUDE' in col][0]

print(f"\nStation data columns:")
print(f"  Name: {station_name_col}")
print(f"  Latitude: {lat_col}")
print(f"  Longitude: {long_col}")

Loading datasets...
✓ Loaded 73 station records
✓ Loaded 74 ridership records

Station data columns:
  Name: features__attributes__PT_NAME
  Latitude: features__attributes__LATITUDE
  Longitude: features__attributes__LONGITUDE


## 5. Prepare Stations Dataset

In [69]:
print("Preparing stations dataset...\n")

# Extract relevant columns
stations_clean = stations_df[[station_name_col, lat_col, long_col]].copy()
stations_clean.columns = ['station_name_original', 'latitude', 'longitude']

# Standardize station names
stations_clean['station_name_std'] = stations_clean['station_name_original'].apply(standardize_station_name)

# Remove any nulls
stations_clean = stations_clean.dropna(subset=['station_name_std', 'latitude', 'longitude'])

print(f"Stations dataset prepared: {len(stations_clean)} records")
print(f"\nSample standardized names:")
print(stations_clean[['station_name_original', 'station_name_std']].head(10))

Preparing stations dataset...

Stations dataset prepared: 73 records

Sample standardized names:
  station_name_original station_name_std
0              EGLINTON         EGLINTON
1            DAVISVILLE       DAVISVILLE
2              ST CLAIR         ST CLAIR
3            SUMMERHILL       SUMMERHILL
4              ROSEDALE         ROSEDALE
5           BLOOR-YONGE      BLOOR-YONGE
6                WARDEN           WARDEN
7                   BAY              BAY
8         ST CLAIR WEST    ST CLAIR WEST
9             ST GEORGE        ST GEORGE


## 6. Prepare Ridership Dataset

In [70]:
print("Preparing ridership dataset...\n")

# Extract relevant columns
ridership_clean = ridership_df[['Station', 'Line', 'Average Weekday Boardings']].copy()
ridership_clean.columns = ['station_name_original', 'line', 'ridership']

# Standardize station names
ridership_clean['station_name_std'] = ridership_clean['station_name_original'].apply(standardize_station_name)

print(f"Ridership dataset prepared: {len(ridership_clean)} records")
print(f"\nSample standardized names:")
print(ridership_clean[['station_name_original', 'station_name_std', 'line', 'ridership']].head(10))

Preparing ridership dataset...

Ridership dataset prepared: 74 records

Sample standardized names:
  station_name_original station_name_std                     line  ridership
0           Bloor-Yonge      BLOOR-YONGE  Line 1 Yonge-University     156643
1               College          COLLEGE  Line 1 Yonge-University      39137
2            Davisville       DAVISVILLE  Line 1 Yonge-University      15903
3        Downsview Park   DOWNSVIEW PARK  Line 1 Yonge-University       5618
4                Dundas           DUNDAS  Line 1 Yonge-University      72406
5                Dupont           DUPONT  Line 1 Yonge-University      11084
6              Eglinton         EGLINTON  Line 1 Yonge-University      60814
7         Eglinton West    EGLINTON WEST  Line 1 Yonge-University      13982
8                 Finch            FINCH  Line 1 Yonge-University      70775
9            Finch West       FINCH WEST  Line 1 Yonge-University      18345


## 7. Handle Multi-Line Stations (Sum Ridership)

In [71]:
print("Handling multi-line stations...\n")

# Identify stations appearing on multiple lines
multi_line_stations = ridership_clean['station_name_std'].value_counts()
multi_line_stations = multi_line_stations[multi_line_stations > 1]

print(f"Stations appearing on multiple lines: {len(multi_line_stations)}")
if len(multi_line_stations) > 0:
    print("\nMulti-line stations (with line breakdown):")
    for station_name in multi_line_stations.index:
        station_entries = ridership_clean[ridership_clean['station_name_std'] == station_name]
        print(f"\n  {station_name}:")
        for _, row in station_entries.iterrows():
            print(f"    - {row['line']}: {row['ridership']:,.0f} boardings")
        total = station_entries['ridership'].sum()
        print(f"    → TOTAL: {total:,.0f} boardings")

# Aggregate ridership by station (sum across lines)
ridership_aggregated = ridership_clean.groupby('station_name_std').agg({
    'ridership': 'sum',
    'line': lambda x: ' & '.join(sorted(set(x)))  # Combine line names
}).reset_index()

# Keep original name from first occurrence
first_names = ridership_clean.groupby('station_name_std')['station_name_original'].first().reset_index()
ridership_aggregated = ridership_aggregated.merge(first_names, on='station_name_std')

print(f"\n✓ Aggregated ridership: {len(ridership_aggregated)} unique stations")
print(f"  (Reduced from {len(ridership_clean)} line-specific entries)")

Handling multi-line stations...

Stations appearing on multiple lines: 4

Multi-line stations (with line breakdown):

  BLOOR-YONGE:
    - Line 1 Yonge-University: 156,643 boardings
    - Line 2 Bloor-Danforth: 121,531 boardings
    → TOTAL: 278,174 boardings

  SHEPPARD-YONGE:
    - Line 1 Yonge-University: 57,501 boardings
    - Line 4 Sheppard: 35,327 boardings
    → TOTAL: 92,828 boardings

  SPADINA:
    - Line 1 Yonge-University: 11,479 boardings
    - Line 2 Bloor-Danforth: 27,601 boardings
    → TOTAL: 39,080 boardings

  ST GEORGE:
    - Line 1 Yonge-University: 101,128 boardings
    - Line 2 Bloor-Danforth: 108,866 boardings
    → TOTAL: 209,994 boardings

✓ Aggregated ridership: 70 unique stations
  (Reduced from 74 line-specific entries)


## 8. Exact Matching (Station Name Join)

In [72]:
print("Performing exact name matching...\n")

# Merge on standardized names
master_exact = stations_clean.merge(
    ridership_aggregated,
    on='station_name_std',
    how='outer',
    suffixes=('_stations', '_ridership'),
    indicator=True
)

# Analyze matching results
match_stats = master_exact['_merge'].value_counts()
print("Exact Matching Results:")
print(f"  Both datasets (matched): {match_stats.get('both', 0)}")
print(f"  Only in stations.csv: {match_stats.get('left_only', 0)}")
print(f"  Only in ridership.csv: {match_stats.get('right_only', 0)}")

# Separate matched and unmatched
matched = master_exact[master_exact['_merge'] == 'both'].copy()
unmatched_stations = master_exact[master_exact['_merge'] == 'left_only'].copy()
unmatched_ridership = master_exact[master_exact['_merge'] == 'right_only'].copy()

print(f"\n✓ Exact matches: {len(matched)} stations")
print(f"⚠️  Unmatched from stations.csv: {len(unmatched_stations)}")
print(f"⚠️  Unmatched from ridership.csv: {len(unmatched_ridership)}")

Performing exact name matching...

Exact Matching Results:
  Both datasets (matched): 68
  Only in stations.csv: 5
  Only in ridership.csv: 2

✓ Exact matches: 68 stations
⚠️  Unmatched from stations.csv: 5
⚠️  Unmatched from ridership.csv: 2


## 9. Fuzzy Matching for Unmatched Stations

In [73]:
print("Performing fuzzy matching for unmatched stations...\n")

fuzzy_matches = []

# For each unmatched ridership station, try to find a match in unmatched stations
if len(unmatched_ridership) > 0 and len(unmatched_stations) > 0:
    ridership_names = unmatched_ridership['station_name_std'].tolist()
    stations_names = unmatched_stations['station_name_std'].tolist()
    
    print("Unmatched stations to resolve:")
    print(f"\nFrom ridership.csv ({len(ridership_names)}):")
    for name in ridership_names:
        print(f"  - {name}")
    
    print(f"\nFrom stations.csv ({len(stations_names)}):")
    for name in stations_names:
        print(f"  - {name}")
    
    print("\n" + "="*80)
    print("Attempting fuzzy matching...")
    print("="*80 + "\n")
    
    for ridership_name in ridership_names:
        best_match, score = find_best_match(ridership_name, stations_names)
        
        if best_match:
            print(f"✓ MATCH FOUND:")
            print(f"  Ridership: '{ridership_name}'")
            print(f"  Stations:  '{best_match}'")
            print(f"  Similarity: {score:.2%}\n")
            
            # Get the full records
            ridership_record = unmatched_ridership[unmatched_ridership['station_name_std'] == ridership_name].iloc[0]
            station_record = unmatched_stations[unmatched_stations['station_name_std'] == best_match].iloc[0]
            
            # Merge
            fuzzy_matches.append({
                'station_name_std': best_match,  # Use stations.csv name as canonical
                'station_name_original_stations': station_record['station_name_original'],
                'station_name_original_ridership': ridership_record['station_name_original_ridership'],
                'latitude': station_record['latitude'],
                'longitude': station_record['longitude'],
                'ridership': ridership_record['ridership'],
                'line': ridership_record['line'],
                'match_type': 'fuzzy',
                'match_score': score
            })
        else:
            print(f"✗ NO MATCH: '{ridership_name}' (best score: {score:.2%})\n")

print(f"\n✓ Fuzzy matching complete: {len(fuzzy_matches)} additional matches found")

Performing fuzzy matching for unmatched stations...

Unmatched stations to resolve:

From ridership.csv (2):
  - HIGHWAY 407
  - VAUGHAN METROPOLITAN CENTRE

From stations.csv (5):
  - ELLESMERE
  - LAWRENCE EAST
  - MCCOWAN
  - MIDLAND
  - SCARBOROUGH CENTRE

Attempting fuzzy matching...

✗ NO MATCH: 'HIGHWAY 407' (best score: 22.22%)

✗ NO MATCH: 'VAUGHAN METROPOLITAN CENTRE' (best score: 48.89%)


✓ Fuzzy matching complete: 0 additional matches found


## 10. Combine All Matches into Master List

In [74]:
print("Creating master station list...\n")

# Prepare exact matches
exact_master = matched[[
    'station_name_std',
    'station_name_original_stations',
    'latitude',
    'longitude',
    'ridership',
    'line'
]].copy()
exact_master['match_type'] = 'exact'
exact_master['match_score'] = 1.0

# Combine exact and fuzzy matches
if len(fuzzy_matches) > 0:
    fuzzy_df = pd.DataFrame(fuzzy_matches)
    fuzzy_df = fuzzy_df[[
        'station_name_std',
        'station_name_original_stations',
        'latitude',
        'longitude',
        'ridership',
        'line',
        'match_type',
        'match_score'
    ]]
    master_list = pd.concat([exact_master, fuzzy_df], ignore_index=True)
else:
    master_list = exact_master.copy()

# Add stations without ridership data (still need coordinates for spatial analysis)
remaining_stations = unmatched_stations[
    ~unmatched_stations['station_name_std'].isin(
        [m['station_name_std'] for m in fuzzy_matches]
    )
].copy()

if len(remaining_stations) > 0:
    print(f"⚠️  Adding {len(remaining_stations)} stations without ridership data:")
    for _, row in remaining_stations.iterrows():
        print(f"  - {row['station_name_std']}")
    
    # FIX: Use the correct column name from unmatched_stations dataframe
    remaining_df = remaining_stations[[
        'station_name_std',
        'station_name_original_stations',  # This column already exists from the merge
        'latitude',
        'longitude'
    ]].copy()
    
    remaining_df['ridership'] = np.nan
    remaining_df['line'] = 'Unknown'
    remaining_df['match_type'] = 'no_ridership'
    remaining_df['match_score'] = np.nan
    
    master_list = pd.concat([master_list, remaining_df], ignore_index=True)

# Rename final column
master_list.rename(columns={
    'station_name_std': 'station_name',
    'ridership': 'total_ridership'
}, inplace=True)

print(f"\n✓ Master list created: {len(master_list)} stations")
print(f"\nBreakdown by match type:")
print(master_list['match_type'].value_counts())

Creating master station list...

⚠️  Adding 5 stations without ridership data:
  - ELLESMERE
  - LAWRENCE EAST
  - MCCOWAN
  - MIDLAND
  - SCARBOROUGH CENTRE

✓ Master list created: 73 stations

Breakdown by match type:
match_type
exact           68
no_ridership     5
Name: count, dtype: int64


## 11. Calculate Distances to Key Venues

In [75]:
print("Calculating distances to key venues...\n")

# Calculate distance to each venue
for venue_key, venue_info in VENUES.items():
    venue_lat = venue_info['lat']
    venue_lon = venue_info['lon']
    venue_name = venue_info['name']
    
    column_name = f'distance_to_{venue_key.lower()}'
    
    # Calculate distance for each station
    master_list[column_name] = master_list.apply(
        lambda row: haversine_distance(
            row['latitude'], 
            row['longitude'], 
            venue_lat, 
            venue_lon
        ),
        axis=1
    )
    
    print(f"✓ Calculated distances to {venue_name}")
    
    # Show closest stations
    closest = master_list.nsmallest(5, column_name)[['station_name', column_name]]
    print(f"  Closest 5 stations:")
    for idx, row in closest.iterrows():
        print(f"    {row['station_name']}: {row[column_name]:.2f} km")
    print()

# Rename distance columns to match spec
master_list.rename(columns={
    'distance_to_bmo_field': 'distance_to_bmo',
    'distance_to_scotiabank_arena': 'distance_to_scotiabank',
    'distance_to_rogers_centre': 'distance_to_rogers'
}, inplace=True)

Calculating distances to key venues...

✓ Calculated distances to BMO Field (Exhibition Place)
  Closest 5 stations:
    ST ANDREW: 3.14 km
    OSGOODE: 3.23 km
    DUFFERIN: 3.28 km
    OSSINGTON: 3.30 km
    CHRISTIE: 3.44 km

✓ Calculated distances to Scotiabank Arena
  Closest 5 stations:
    UNION: 0.25 km
    KING: 0.64 km
    ST ANDREW: 0.66 km
    QUEEN: 1.00 km
    OSGOODE: 1.01 km

✓ Calculated distances to Rogers Centre
  Closest 5 stations:
    ST ANDREW: 0.76 km
    UNION: 0.95 km
    OSGOODE: 1.06 km
    KING: 1.28 km
    ST PATRICK: 1.50 km



## 12. Flag Stations Near Venues (Within 2km)

In [76]:
print(f"Flagging stations within {PROXIMITY_THRESHOLD_KM} km of venues...\n")

# Flag if station is near ANY venue
master_list['is_near_venue'] = (
    (master_list['distance_to_bmo'] <= PROXIMITY_THRESHOLD_KM) |
    (master_list['distance_to_scotiabank'] <= PROXIMITY_THRESHOLD_KM) |
    (master_list['distance_to_rogers'] <= PROXIMITY_THRESHOLD_KM)
)

# Individual venue flags (for detailed analysis)
master_list['is_near_bmo'] = master_list['distance_to_bmo'] <= PROXIMITY_THRESHOLD_KM
master_list['is_near_scotiabank'] = master_list['distance_to_scotiabank'] <= PROXIMITY_THRESHOLD_KM
master_list['is_near_rogers'] = master_list['distance_to_rogers'] <= PROXIMITY_THRESHOLD_KM

# Summary
near_venue_count = master_list['is_near_venue'].sum()
print(f"✓ Stations near at least one venue: {near_venue_count}")

print(f"\nBreakdown by venue:")
print(f"  Near BMO Field: {master_list['is_near_bmo'].sum()}")
print(f"  Near Scotiabank Arena: {master_list['is_near_scotiabank'].sum()}")
print(f"  Near Rogers Centre: {master_list['is_near_rogers'].sum()}")

# Show stations near venues
if near_venue_count > 0:
    print(f"\nStations near venues (≤{PROXIMITY_THRESHOLD_KM} km):")
    near_stations = master_list[master_list['is_near_venue']].copy()
    near_stations_display = near_stations[[
        'station_name',
        'distance_to_bmo',
        'distance_to_scotiabank',
        'distance_to_rogers',
        'is_near_bmo',
        'is_near_scotiabank',
        'is_near_rogers'
    ]].sort_values('distance_to_bmo')
    
    print(near_stations_display.to_string(index=False))

Flagging stations within 2.0 km of venues...

✓ Stations near at least one venue: 8

Breakdown by venue:
  Near BMO Field: 0
  Near Scotiabank Arena: 8
  Near Rogers Centre: 7

Stations near venues (≤2.0 km):
station_name  distance_to_bmo  distance_to_scotiabank  distance_to_rogers  is_near_bmo  is_near_scotiabank  is_near_rogers
   ST ANDREW           3.1444                  0.6568              0.7553        False                True            True
     OSGOODE           3.2336                  1.0143              1.0561        False                True            True
  ST PATRICK           3.4569                  1.4490              1.4961        False                True            True
       UNION           3.4845                  0.2517              0.9513        False                True            True
        KING           3.7708                  0.6370              1.2843        False                True            True
       QUEEN           3.9192                  1.0006

## 13. Quality Checks

In [77]:
print("\n" + "="*80)
print("QUALITY CHECKS")
print("="*80 + "\n")

issues = []
warnings = []
passed = []

# Check 1: All stations have coordinates
missing_coords = master_list[master_list['latitude'].isna() | master_list['longitude'].isna()]
if len(missing_coords) == 0:
    passed.append("✓ All stations have valid coordinates")
else:
    issues.append(f"✗ {len(missing_coords)} stations missing coordinates")
    print(f"Stations missing coordinates:")
    print(missing_coords[['station_name', 'latitude', 'longitude']])

# Check 2: Expected station count
expected_min = 73  # From stations.csv
expected_max = 75  # Allowing for minor variations
actual_count = len(master_list)

if expected_min <= actual_count <= expected_max:
    passed.append(f"✓ Station count in expected range: {actual_count} (expected {expected_min}-{expected_max})")
elif actual_count < expected_min:
    warnings.append(f"⚠️  Fewer stations than expected: {actual_count} (expected {expected_min}+)")
else:
    warnings.append(f"⚠️  More stations than expected: {actual_count} (expected {expected_max} max)")

# Check 3: Ridership data coverage
with_ridership = master_list['total_ridership'].notna().sum()
ridership_pct = with_ridership / len(master_list) * 100

if ridership_pct >= 90:
    passed.append(f"✓ Ridership data coverage: {with_ridership}/{len(master_list)} ({ridership_pct:.1f}%)")
elif ridership_pct >= 80:
    warnings.append(f"⚠️  Ridership coverage acceptable but incomplete: {ridership_pct:.1f}%")
else:
    issues.append(f"✗ Poor ridership coverage: {ridership_pct:.1f}%")

# Check 4: Distance calculations
missing_distances = master_list[
    master_list['distance_to_bmo'].isna() |
    master_list['distance_to_scotiabank'].isna() |
    master_list['distance_to_rogers'].isna()
]

if len(missing_distances) == 0:
    passed.append("✓ All stations have distance calculations")
else:
    issues.append(f"✗ {len(missing_distances)} stations missing distance calculations")

# Check 5: Reasonable distance ranges
max_distance = master_list[['distance_to_bmo', 'distance_to_scotiabank', 'distance_to_rogers']].max().max()
if max_distance <= 50:  # Toronto is ~40km wide
    passed.append(f"✓ All distances within reasonable range (max: {max_distance:.1f} km)")
else:
    warnings.append(f"⚠️  Some distances seem large (max: {max_distance:.1f} km) - verify coordinates")

# Check 6: Near venue flags
if near_venue_count > 0:
    passed.append(f"✓ Proximity flags set: {near_venue_count} stations near venues")
else:
    warnings.append("⚠️  No stations flagged as near venues (threshold may be too strict)")

# Print results
print("PASSED CHECKS:")
for item in passed:
    print(f"  {item}")

if warnings:
    print(f"\nWARNINGS:")
    for item in warnings:
        print(f"  {item}")

if issues:
    print(f"\nISSUES:")
    for item in issues:
        print(f"  {item}")

# Overall assessment
print(f"\n" + "="*80)
if len(issues) == 0:
    status = "✓✓✓ EXCELLENT"
    message = "Master station list is ready for spatial analysis."
elif len(issues) <= 1 and len(warnings) <= 2:
    status = "✓ GOOD"
    message = "Master station list is acceptable with minor caveats."
else:
    status = "⚠️  NEEDS REVIEW"
    message = "Review issues before proceeding."

print(f"QUALITY ASSESSMENT: {status}")
print(f"{message}")
print("="*80)


QUALITY CHECKS

PASSED CHECKS:
  ✓ All stations have valid coordinates
  ✓ Station count in expected range: 73 (expected 73-75)
  ✓ Ridership data coverage: 68/73 (93.2%)
  ✓ All stations have distance calculations
  ✓ All distances within reasonable range (max: 20.7 km)
  ✓ Proximity flags set: 8 stations near venues

QUALITY ASSESSMENT: ✓✓✓ EXCELLENT
Master station list is ready for spatial analysis.


## 14. Final Dataset Preparation

In [78]:
print("Preparing final dataset...\n")

# Select and order columns as specified
final_columns = [
    'station_name',
    'latitude',
    'longitude',
    'total_ridership',
    'line',
    'distance_to_bmo',
    'distance_to_scotiabank',
    'distance_to_rogers',
    'is_near_venue',
    'is_near_bmo',
    'is_near_scotiabank',
    'is_near_rogers'
]

master_final = master_list[final_columns].copy()

# Sort by ridership (descending) for easier reference
master_final = master_final.sort_values('total_ridership', ascending=False, na_position='last')

# Reset index
master_final = master_final.reset_index(drop=True)

print(f"✓ Final dataset prepared: {len(master_final)} stations")
print(f"\nColumn order:")
for i, col in enumerate(master_final.columns, 1):
    print(f"  {i}. {col}")

print(f"\nTop 10 stations by ridership:")
print(master_final[['station_name', 'total_ridership', 'line']].head(10))

Preparing final dataset...

✓ Final dataset prepared: 73 stations

Column order:
  1. station_name
  2. latitude
  3. longitude
  4. total_ridership
  5. line
  6. distance_to_bmo
  7. distance_to_scotiabank
  8. distance_to_rogers
  9. is_near_venue
  10. is_near_bmo
  11. is_near_scotiabank
  12. is_near_rogers

Top 10 stations by ridership:
     station_name  total_ridership  \
0     BLOOR-YONGE      278174.0000   
1       ST GEORGE      209994.0000   
2           UNION      136515.0000   
3  SHEPPARD-YONGE       92828.0000   
4          DUNDAS       72406.0000   
5           FINCH       70775.0000   
6        EGLINTON       60814.0000   
7         KIPLING       49392.0000   
8         KENNEDY       42881.0000   
9         COLLEGE       39137.0000   

                                              line  
0  Line 1 Yonge-University & Line 2 Bloor-Danforth  
1  Line 1 Yonge-University & Line 2 Bloor-Danforth  
2                          Line 1 Yonge-University  
3        Line 1 Yonge-U

## 15. Save Outputs

In [79]:
print("Saving outputs...\n")

# Save master station list
master_final.to_csv(MASTER_STATIONS_PATH, index=False)
print(f"✓ Saved master station list: {MASTER_STATIONS_PATH}")
print(f"  Records: {len(master_final)}")
print(f"  Columns: {len(master_final.columns)}")

# Save unmatched stations report (if any)
unmatched_remaining = []

# Check for remaining unmatched ridership entries
ridership_matched_names = set(master_final['station_name'].unique())
all_ridership_names = set(ridership_aggregated['station_name_std'].unique())
unmatched_ridership_names = all_ridership_names - ridership_matched_names

if len(unmatched_ridership_names) > 0:
    print(f"\n⚠️  Unmatched ridership stations: {len(unmatched_ridership_names)}")
    unmatched_ridership_df = ridership_aggregated[
        ridership_aggregated['station_name_std'].isin(unmatched_ridership_names)
    ]
    unmatched_ridership_df.to_csv(UNMATCHED_STATIONS_PATH, index=False)
    print(f"  Saved to: {UNMATCHED_STATIONS_PATH}")
    for name in sorted(unmatched_ridership_names):
        print(f"    - {name}")

# Generate matching report
report_lines = []
report_lines.append("="*80)
report_lines.append("STATION NAME MATCHING REPORT")
report_lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append("="*80)
report_lines.append("")

report_lines.append("SUMMARY:")
report_lines.append(f"  Input - Stations dataset: {len(stations_clean)} records")
report_lines.append(f"  Input - Ridership dataset: {len(ridership_clean)} records")
report_lines.append(f"  Input - Unique ridership stations: {len(ridership_aggregated)} (after aggregating multi-line)")
report_lines.append(f"  Output - Master list: {len(master_final)} stations")
report_lines.append("")

report_lines.append("MATCHING BREAKDOWN:")
report_lines.append(f"  Exact matches: {(master_list['match_type'] == 'exact').sum()}")
report_lines.append(f"  Fuzzy matches: {(master_list['match_type'] == 'fuzzy').sum()}")
report_lines.append(f"  Stations without ridership: {(master_list['match_type'] == 'no_ridership').sum()}")
report_lines.append("")

report_lines.append("FUZZY MATCHES DETAIL:")
if len(fuzzy_matches) > 0:
    for match in fuzzy_matches:
        report_lines.append(f"  - Ridership: '{match['station_name_original_ridership']}'")
        report_lines.append(f"    Stations:  '{match['station_name_original_stations']}'")
        report_lines.append(f"    Score: {match['match_score']:.2%}")
        report_lines.append("")
else:
    report_lines.append("  (None)")
    report_lines.append("")

report_lines.append("VENUE PROXIMITY:")
report_lines.append(f"  Threshold: {PROXIMITY_THRESHOLD_KM} km")
report_lines.append(f"  Stations near any venue: {near_venue_count}")
report_lines.append(f"  Near BMO Field: {master_final['is_near_bmo'].sum()}")
report_lines.append(f"  Near Scotiabank Arena: {master_final['is_near_scotiabank'].sum()}")
report_lines.append(f"  Near Rogers Centre: {master_final['is_near_rogers'].sum()}")
report_lines.append("")

report_lines.append("DATA QUALITY:")
report_lines.append(f"  Stations with coordinates: {master_final['latitude'].notna().sum()}")
report_lines.append(f"  Stations with ridership: {master_final['total_ridership'].notna().sum()}")
report_lines.append(f"  Ridership coverage: {master_final['total_ridership'].notna().sum() / len(master_final) * 100:.1f}%")
report_lines.append("")

report_lines.append("TOP 10 STATIONS BY RIDERSHIP:")
for idx, row in master_final.head(10).iterrows():
    ridership_str = f"{row['total_ridership']:,.0f}" if pd.notna(row['total_ridership']) else "N/A"
    report_lines.append(f"  {idx+1}. {row['station_name']}: {ridership_str} boardings/day")
report_lines.append("")

report_lines.append("="*80)
report_lines.append("END OF REPORT")
report_lines.append("="*80)

# Save report
with open(MATCHING_REPORT_PATH, 'w') as f:
    f.write('\n'.join(report_lines))

print(f"\n✓ Saved matching report: {MATCHING_REPORT_PATH}")
print(f"\n{'='*80}")
print("PROMPT 2 COMPLETE")
print(f"{'='*80}")

Saving outputs...

✓ Saved master station list: /Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/TPS_CaseComp/outputs/02_master_station_list.csv
  Records: 73
  Columns: 12

⚠️  Unmatched ridership stations: 2
  Saved to: /Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/TPS_CaseComp/outputs/02_unmatched_stations.csv
    - HIGHWAY 407
    - VAUGHAN METROPOLITAN CENTRE

✓ Saved matching report: /Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/TPS_CaseComp/outputs/02_name_matching_report.txt

PROMPT 2 COMPLETE


## 16. Summary Statistics

In [80]:
print("\nFINAL SUMMARY STATISTICS:")
print("="*80)

print(f"\nMaster Station List:")
print(f"  Total stations: {len(master_final)}")
print(f"  With ridership data: {master_final['total_ridership'].notna().sum()}")
print(f"  Without ridership data: {master_final['total_ridership'].isna().sum()}")

print(f"\nRidership Statistics:")
print(master_final['total_ridership'].describe())

print(f"\nDistance Statistics (km):")
distance_stats = master_final[['distance_to_bmo', 'distance_to_scotiabank', 'distance_to_rogers']].describe()
print(distance_stats)

print(f"\nVenue Proximity Summary:")
print(f"  Stations ≤{PROXIMITY_THRESHOLD_KM}km from any venue: {master_final['is_near_venue'].sum()}")
print(f"  Stations ≤{PROXIMITY_THRESHOLD_KM}km from BMO Field: {master_final['is_near_bmo'].sum()}")
print(f"  Stations ≤{PROXIMITY_THRESHOLD_KM}km from Scotiabank: {master_final['is_near_scotiabank'].sum()}")
print(f"  Stations ≤{PROXIMITY_THRESHOLD_KM}km from Rogers Centre: {master_final['is_near_rogers'].sum()}")

print(f"\n{'='*80}")
print("Ready for Prompt 3: Spatial Join (Crimes → Stations)")
print(f"{'='*80}")


FINAL SUMMARY STATISTICS:

Master Station List:
  Total stations: 73
  With ridership data: 68
  Without ridership data: 5

Ridership Statistics:
count       68.0000
mean     30992.1029
std      43591.4799
min       3180.0000
25%      11641.7500
50%      19733.5000
75%      31017.5000
max     278174.0000
Name: total_ridership, dtype: float64

Distance Statistics (km):
       distance_to_bmo  distance_to_scotiabank  distance_to_rogers
count          73.0000                 73.0000             73.0000
mean            8.9779                  8.0214              8.0905
std             5.1984                  5.1604              5.1792
min             3.1444                  0.2517              0.7553
25%             4.2804                  3.7982              3.7766
50%             7.4537                  7.0493              7.0600
75%            13.4475                 12.5064             11.7935
max            20.6959                 18.2262             18.5033

Venue Proximity Summary:

---

## Key Outputs Summary

### Master Station List
- **File:** `02_master_station_list.csv`
- **Records:** ~73-75 stations (exact vs fuzzy vs no-ridership)
- **Columns:** 12 fields including coordinates, ridership, distances, proximity flags

### Name Matching
- **Exact matches:** Stations where names matched perfectly after standardization
- **Fuzzy matches:** Stations matched using similarity algorithm (≥85% threshold)
- **Unmatched:** Stations in one dataset but not the other

### Venue Proximity
- **BMO Field:** Exhibition Place (FIFA 2026 venue)
- **Scotiabank Arena:** Major concerts and sports events
- **Rogers Centre:** Blue Jays games and concerts
- **Threshold:** 2km radius

### Next Steps
1. Use master station list for spatial join with crime data (Prompt 3)
2. Venue proximity flags will identify FIFA-critical stations
3. Ridership data will weight risk assessments

---